In [92]:
import os
import json
import re
import random
from tqdm import tqdm

os.environ['OPENAI_API_KEY'] = ""

In [5]:
from openai import OpenAI


client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
    logprobs=True, top_logprobs=5,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [78]:
str(rubq_preds)

"{'P155': 'предыдущее по порядку', 'P1830': 'владеет', 'P2579': 'изучается дисциплиной', 'P569': 'др', 'P1365': 'заменил', 'P576': 'дата прекращения существования', 'P1884': 'цвет волос', 'P885': 'источник воды', 'P25': 'мать', 'P1303': 'музыкальный инструмент', 'P31': 'это частный случай понятия', 'P5096': 'член экипажа', 'P676': 'автор слов', 'P176': 'изготовитель', 'P279': 'гипоним от', 'P156': 'next', 'P1346': 'победительница', 'P931': 'обслуживаемый город', 'P84': 'архитектор', 'P131': 'административно-территориальная единица', 'P108': 'работодатель', 'P274': 'химическая формула', 'P509': 'метод убийства', 'P2047': 'длительность', 'P823': 'выступающая', 'P2053': 'бассейн водоёма', 'P1448': 'официальное название', 'P1344': 'принял(а) участие в', 'P522': 'тип орбиты', 'P2079': 'изготавливается посредством или с помощью', 'P2551': 'использованный метр', 'P36': 'столица', 'P921': 'основная тема', 'P642': 'относится к', 'P106': 'род занятий', 'P17': 'государство', 'P571': 'дата основан

In [184]:
rubq_train = json.load(open("/Users/somov-od/Documents/phd/projects/open_kgqa/data/RuBQ/RuBQ_2.0/train_with_aliases.json", 'r'))
rubq_test = json.load(open("/Users/somov-od/Documents/phd/projects/open_kgqa/data/RuBQ/RuBQ_2.0/test_with_aliases.json", 'r'))
rubq_preds = json.load(open("/Users/somov-od/Documents/phd/projects/open_kgqa/data/RuBQ/RuBQ_2.0/rubq_english_preds_description.json", 'r'))
len(rubq_train), len(rubq_test), len(rubq_preds)

(1691, 419, 277)

In [142]:
rubq_preds

{'wdt:P155': 'предыдущее по порядку',
 'wdt:P1830': 'владеет',
 'wdt:P2579': 'изучается дисциплиной',
 'wdt:P569': 'др',
 'wdt:P1365': 'заменил',
 'wdt:P576': 'дата прекращения существования',
 'wdt:P1884': 'цвет волос',
 'wdt:P885': 'источник воды',
 'wdt:P25': 'мать',
 'wdt:P1303': 'музыкальный инструмент',
 'wdt:P31': 'это частный случай понятия',
 'wdt:P5096': 'член экипажа',
 'wdt:P676': 'автор слов',
 'wdt:P176': 'изготовитель',
 'wdt:P279': 'гипоним от',
 'wdt:P156': 'next',
 'wdt:P1346': 'победительница',
 'wdt:P931': 'обслуживаемый город',
 'wdt:P84': 'архитектор',
 'wdt:P131': 'административно-территориальная единица',
 'wdt:P108': 'работодатель',
 'wdt:P274': 'химическая формула',
 'wdt:P509': 'метод убийства',
 'wdt:P2047': 'длительность',
 'wdt:P823': 'выступающая',
 'wdt:P2053': 'бассейн водоёма',
 'wdt:P1448': 'официальное название',
 'wdt:P1344': 'принял(а) участие в',
 'wdt:P522': 'тип орбиты',
 'wdt:P2079': 'изготавливается посредством или с помощью',
 'wdt:P2551': 'и

In [66]:
appropriate_rubq_test_samples = []
for sample in rubq_test:
    if sample['query'] and len(sample['id2alias'])>0:
        appropriate_rubq_test_samples.append(sample)
        
print(len(appropriate_rubq_test_samples))

chat_gpt_test_subsample = random.sample(appropriate_rubq_test_samples, 100)
len(chat_gpt_test_subsample)

# check
for sample in chat_gpt_test_subsample:
    if sample['query'] and len(sample['id2alias'])>0:
        continue
    else:
        print('Error')
        break
print('All good samples')

391
All good samples


In [158]:
# lets look of the templates in input

def generate_template(sparql):
    predicates = re.findall(r"\S*:P\d+(?:\+|\*)?", sparql)
    ids = re.findall(r"\S*Q\d+", sparql)

    for idx, pred in enumerate(predicates):
        sparql = sparql.replace(pred, f"PRED_{idx}")

    for idx, wikidata_id in enumerate(ids):
        sparql = sparql.replace(wikidata_id, f"ENTITY_{idx}")

    sparql = sparql.replace('\n', " ")
    return sparql

rubq_train_templates = set()
for sample in rubq_train:
    sparql = sample['query']
    if sparql:
        sparql_template = generate_template(sparql)
        rubq_train_templates.add(sparql_template)
        
        
rubq_test_templates = set()
for sample in rubq_test:
    sparql = sample['query']
    if sparql:
        sparql_template = generate_template(sparql)
        rubq_test_templates.add(sparql_template)

        
print('Total RUBQ train templates: ', len(rubq_train_templates))
print('Total RUBQ test templates: ', len(rubq_test_templates))
print('Total RUBQ test templates: ', len(rubq_train_templates.intersection(rubq_test_templates)))

Total RUBQ train templates:  100
Total RUBQ test templates:  36
Total RUBQ test templates:  26


In [187]:
rubq_preds

{'wdt:P403': 'the body of water to which the watercourse drains',
 'wdt:P559': 'the feature (intersecting road, train station, etc.) at the end of a linear feature',
 'wdt:P88': 'person or organization that commissioned this work',
 'wdt:P1366': 'other person or item which continues the item by replacing it in its role. Use P156 ("followed by") if the item is not replaced nor identical, but adds to the series (e.g. books in a series).',
 'wdt:P706': 'located on the specified (geo)physical feature. Should not be used when the value is only political/administrative (P131) or a mountain range (P4552).',
 'pq:P585': 'date something took place, existed or a statement was true; for providing time use the "refine date" property (P4241)',
 'wdt:P942': 'the theme music/song used by the item',
 'p:P69': 'educational institution attended by subject',
 'p:P26': 'the subject has the object as their spouse (husband, wife, partner, etc.). Use "unmarried partner" (P451) for non-married companions',
 '

In [160]:
rubq_templates = "\n".join(rubq_tempalates)
instruction = f"""You are a text-to-SPARQL assistant. You can generate the following SPARQL structures: 
{rubq_templates}

You know the following predicates that can be used in SPARQL:
{str(rubq_preds)}
Generate a only SPARQL query based on user question Q and given question entities, matched to Wikidata Ids, mentioned as aliases A. 
Generate SPARQL from a given templates or generate novel grammarly correct SPARQL. 
Use only provided predicates and only provided entities to generate query. 
Do not generate anything besides SPARQL query."""

In [153]:
entity_alias

"{'wd:Q1710170': {'en': ['Tower of Kamyanyets']}}"

In [152]:
sample = chat_gpt_test_subsample[0]
question = sample['question_eng']
entity_alias = str({f"wd:{wd_id}":value for wd_id, value in sample['id2alias'].items()})
user_question = f"Q: {question}, Entity aliases: {entity_alias}"

completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": instruction},
    {"role": "user", "content": user_question}
  ]
)

completion = completion.choices[0].message.content

completion

'```sparql\nSELECT ?answer WHERE { wd:Q1710170 wdt:P149 ?answer . }\n```'

In [155]:

predicated_requests = []
for sample in tqdm(chat_gpt_test_subsample[:]):
    question = sample['question_eng']
    entity_alias = str({f"wd:{wd_id}":value for wd_id, value in sample['id2alias'].items()})
    
    user_question = f"Q: {question}, Entity aliases: {entity_alias}"

    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": user_question}
      ]
    )

    completion = completion.choices[0].message.content
    
    predicated_requests.append(completion)

100%|██████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:48<00:00,  2.29s/it]


In [161]:
def clean_sparql_query(query):
    # Remove leading and trailing whitespace and newlines
    query = query.strip()
    query = query.replace('sparql', "")
    query = query.replace("```", "").strip()
    # Replace multiple whitespaces, newlines, and tabs with a single space
    query = re.sub(r'\s+', ' ', query)
    # Ensure proper spacing around curly braces
    query = re.sub(r'\s*{\s*', ' { ', query)
    query = re.sub(r'\s*}\s*', ' } ', query)
    query = query.replace('.', "")
    query = list(filter(lambda x: x != '', query.split(" ")))
    query = " ".join(query)
    return query

# Example usage:
query = '```sparql SELECT ?answer WHERE {  wd:Q1710170 wdt:P149 ?answer . }```'
clean_query = clean_sparql_query(query)
print(clean_query)

SELECT ?answer WHERE { wd:Q1710170 wdt:P149 ?answer }


In [163]:
def compare_sparql_queries(query1, query2):
    # Normalize both queries
    normalized_query1 = clean_sparql_query(query1)
    normalized_query2 = clean_sparql_query(query2)
    # Compare normalized queries for exact match
    return normalized_query1 == normalized_query2

In [165]:
golds = [sample['query'] for sample in chat_gpt_test_subsample[:]]

exact_match = 0
for pred, gold in zip(predicated_requests, golds):
    result = compare_sparql_queries(pred, gold)
    if result == False:
        # print(pred)
        # print(gold)
        print('Pred: ', clean_sparql_query(pred))
        print('Gold: ', clean_sparql_query(gold))
        print()
    exact_match += result
    
print('Exact Match score: ', exact_match / len(golds))

Pred:  SELECT ?answer WHERE { wd:Q179444 wdt:P206 ?answer ?answer wdt:P31 wd:Q5119 }
Gold:  SELECT ?answer WHERE { ?answer wdt:P206 wd:Q179444 ?answer wdt:P31 wd:Q5119 }

Pred:  SELECT ?answer WHERE { wd:Q812767 wdt:P17 ?answer }
Gold:  SELECT ?answer WHERE { wd:Q812767 wdt:P740 ?answer }

Pred:  SELECT ?answer WHERE { wd:Q38111 wdt:P1559 ?answer FILTER (lang(?answer) = "en") }
Gold:  SELECT ?answer WHERE { wd:Q38111 wdt:P1477 ?answer }

Pred:  SELECT ?answer WHERE { wd:Q1865 wdt:P17 ?answer }
Gold:  SELECT ?answer WHERE { wd:Q1865 wdt:P1376 ?answer }

Pred:  SELECT ?answer WHERE { ?answer wdt:P170 wd:Q500951 ?answer wdt:P180 wd:Q500951 }
Gold:  SELECT ?answer WHERE { wd:Q500951 wdt:P170 ?answer }

Pred:  SELECT ?answer WHERE { wd:Q34216 rdfs:label ?answer FILTER (lang(?answer) = "en") }
Gold:  SELECT ?answer WHERE { wd:Q34216 rdfs:label ?answer FILTER (lang(?answer) = "ru") }

Pred:  SELECT ?answer WHERE { wd:Q44301 wdt:P1366 ?answer }
Gold:  SELECT ?answer WHERE { wd:Q44301 wdt:P463 

In [173]:
golds

['SELECT ?answer \nWHERE {\n  wd:Q1710170 wdt:P149 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  ?answer wdt:P206 wd:Q179444 .\n  ?answer wdt:P31 wd:Q5119\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q148 wdt:P36 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q170172 wdt:P119 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q812767 wdt:P740 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q30 wdt:P150 ?answer\n}',
 'SELECT ?answer\nWHERE {\n  wd:Q38111 wdt:P1477 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q1865 wdt:P1376 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q203002 wdt:P54 ?answer\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q500951 wdt:P170 ?answer\n}',
 'SELECT ?answer\nWHERE {\n  wd:Q34216 rdfs:label ?answer .\n  FILTER (lang(?answer) = "ru") .\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q44301 wdt:P463 ?answer .\n  ?answer wdt:P31/wdt:P279 wd:Q215380\n}',
 'SELECT ?answer \nWHERE {\n wd:Q151469 wdt:P571 ?answer .\n}',
 'SELECT ?answer \nWHERE {\n  wd:Q148 wdt:P150 ?answer\n}',
 'SELECT ?answer \nWHER

In [178]:
clean_gold_queries

[[...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...]]

In [179]:
import pandas as pd


clean_pred_queries = []
clean_gold_queries = []
exact_match_list = []
for pred, gold in zip(predicated_requests, golds):
    
    cleaned_pred = clean_sparql_query(pred)
    cleaned_gold = clean_sparql_query(gold)
    
    clean_pred_queries.append(cleaned_pred)
    clean_gold_queries.append(cleaned_gold)
    
    exact_match_list.append(cleaned_pred == cleaned_gold)
    
df = pd.DataFrame.from_dict({"gold": clean_gold_queries, "pred": clean_pred_queries, "exact_match": exact_match_list})

In [183]:
df.to_csv("chat_gpt_as_kgqa_sample_11-12-24.tsv", sep='\t')